Import tools.

In [111]:
import pandas as pd
from pickle import load
from pickle import dump
import numpy as np
pd.set_option("max_rows", None)
pd.set_option("max_columns", None)

The goal of this project is to build a model that can predict the winner of presidential elections by county. Data was gathered from several different sources and then preprocessed and joined together. 

Data:
2016 and 2012 election results: https://data.world/garyhoov/2016-pres-election-by-county  
Alaska 2016 and 2012 election results: https://www.thecinyc.com/  
Additional Alaska information: https://en.wikipedia.org/wiki/List_of_boroughs_and_census_areas_in_Alaska  
Race: https://data.census.gov/cedsci/table?q=county%20population&tid=DECENNIALPL2020.P1  
Income: https://data.census.gov/cedsci/all?q=county%20population  

## First Dataset: RESULTS

This dataset includes 2016 Presidential election results by county, 2012 presidential election results by county, county density, county type, and land area. It does not include Alaska. Alaska information can be found in the next section of this notebook.
Data: https://data.world/garyhoov/2016-pres-election-by-county 

Open dataset.

In [112]:
results = pd.read_csv('data/2016 Presidential Election Analysis.csv')

In [113]:
results.head()

,State Code,County Name,County Population,Clinton or Trump State,Clinton,Trump,Total,% Clinton,% Trump,Vote Difference C-T,Vote Difference T-C,Percent Difference C-T,Percent Difference T-C,Obama,Romney,2012 Total Votes,% Obama,% Romney,Vote Difference O-R,% Difference O-R,Clinton B(W) Obama,Trump B(W) Romney,Clinton % B(W) Obama,Trump % B(W) Romney,2010 Land Area,Density,Combined FIPS Code,ANSI Code,State FIPS Code,County FIPS Code,CBSA Code,CSA Code,CBSA Title,CBSA Population,Metropolitan/Micropolitan Statistical Area,CSA Title,Metropolitan Division Title,Metropolitan Division Code,Central/Outlying County
0,AL,Autauga County,"55,347",Trump,"5,908","18,110","24,661",23.96%,73.44%,"(12,202)","12,202",-49.48%,49.48%,"6,354","17,366","23,909",26.58%,72.63%,"(11,012)",-46.06%,(446),744,-2.62%,0.80%,594,93,1001,01001,1.0,1.0,33860.0,NaN,"Montgomery, AL","373,792",Metropolitan Statistical Area,NaN,NaN,NaN,Central
1,AL,Baldwin County,"203,709",Trump,"18,409","72,780","94,090",19.57%,77.35%,"(54,371)","54,371",-57.79%,57.79%,"18,329","65,772","84,988",21.57%,77.39%,"(47,443)",-55.82%,80,"7,008",-2.00%,-0.04%,1590,128,1003,01003,1.0,3.0,19300.0,380.0,"Daphne-Fairhope-Foley, AL","203,709",Metropolitan Statistical Area,"Mobile-Daphne-Fairhope, AL",NaN,NaN,Central
2,AL,Barbour County,"26,489",Trump,"4,848","5,431","10,390",46.66%,52.27%,(583),583,-5.61%,5.61%,"5,873","5,539","11,459",51.25%,48.34%,334,2.91%,"(1,025)",(108),-4.59%,3.93%,885,30,1005,01005,1.0,5.0,21640.0,NaN,"Eufaula, AL-GA",NaN,Micropolitan Statistical Area,NaN,NaN,NaN,Central
3,AL,Bibb County,"22,583",Trump,"1,874","6,733","8,748",21.42%,76.97%,"(4,859)","4,859",-55.54%,55.54%,"2,200","6,131","8,391",26.22%,73.07%,"(3,931)",-46.85%,(326),602,-4.80%,3.90%,623,36,1007,01007,1.0,7.0,13820.0,142.0,"Birmingham-Hoover, AL","1,145,647",Metropolitan Statistical Area,"Birmingham-Hoover-Talladega, AL",NaN,NaN,Outlying
4,AL,Blount County,"57,673",Trump,"2,150","22,808","25,384",8.47%,89.85%,"(20,658)","20,658",-81.38%,81.38%,"2,961","20,741","23,980",12.35%,86.49%,"(17,780)",-74.15%,(811),"2,067",-3.88%,3.36%,645,89,1009,01009,1.0,9.0,13820.0,142.0,"Birmingham-Hoover, AL","1,145,647",Metropolitan Statistical Area,"Birmingham-Hoover-Talladega, AL",NaN,NaN,Outlying


Only keep helpful columns. Keep Clinton and Trump total votes until target is established.

In [114]:
results = results.iloc[:, [0, 1, 4, 5, 6, 13, 14, 15, 24, 25, 38]]

In [115]:
results.head()

,State Code,County Name,Clinton,Trump,Total,Obama,Romney,2012 Total Votes,2010 Land Area,Density,Central/Outlying County
0,AL,Autauga County,"5,908","18,110","24,661","6,354","17,366","23,909",594,93,Central
1,AL,Baldwin County,"18,409","72,780","94,090","18,329","65,772","84,988",1590,128,Central
2,AL,Barbour County,"4,848","5,431","10,390","5,873","5,539","11,459",885,30,Central
3,AL,Bibb County,"1,874","6,733","8,748","2,200","6,131","8,391",623,36,Outlying
4,AL,Blount County,"2,150","22,808","25,384","2,961","20,741","23,980",645,89,Outlying


Remove punctuation from column names and values.

In [116]:
results.columns = results.columns.str.strip().str.replace('[^\w\s]', '')

<ipython-input-116-12a2c89aa82b>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  results.columns = results.columns.str.strip().str.replace('[^\w\s]', '')


In [117]:
def remove_punctuation(x):
    try:
        x = x.str.replace('[^\w\s]','')
    except:
        pass
    return x

In [118]:
results = results.apply(remove_punctuation)

<ipython-input-117-fabae1a16f96>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  x = x.str.replace('[^\w\s]','')


In [119]:
results.head()

,State Code,County Name,Clinton,Trump,Total,Obama,Romney,2012 Total Votes,2010 Land Area,Density,CentralOutlying County
0,AL,Autauga County,5908,18110,24661,6354,17366,23909,594,93,Central
1,AL,Baldwin County,18409,72780,94090,18329,65772,84988,1590,128,Central
2,AL,Barbour County,4848,5431,10390,5873,5539,11459,885,30,Central
3,AL,Bibb County,1874,6733,8748,2200,6131,8391,623,36,Outlying
4,AL,Blount County,2150,22808,25384,2961,20741,23980,645,89,Outlying


Lengthen state names.

In [120]:
us_state_abbrev = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming',
}

In [121]:
results['State Code'] = results['State Code'].map(us_state_abbrev)

Rename columns.

In [122]:
results.rename(columns = {'Total': '2016_total_votes', '2012 Total Votes':'2012_total_votes', 'State Code': 'State', 'County Name': 'County', 'CentralOutlying County': 'central_outlying', '2010 Land Area': '2010_land_area'}, inplace = True)

If state is missing, fill with county name. Pretty sure this only applies to DC.

In [123]:
results['State'] = results['State'].fillna(results['County'])

Alaska is not broken down by county in this dataset. Dropping all Alaska info and pulling in results and info from another source.

In [124]:
results.drop(results.loc[results['State'].str.contains('Alaska', case=False)].index, inplace = True)

Simplify long string to easier to read string.

In [125]:
results.replace('Not Metro or Micro Presumed Rural', 'Rural', inplace = True)

Change datatypes to numeric where necessary.

In [126]:
results.iloc[:, 2:9] = results.iloc[:, 2:9].apply(pd.to_numeric)

In [127]:
results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3112 entries, 0 to 3112
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   State             3112 non-null   object
 1   County            3112 non-null   object
 2   Clinton           3112 non-null   int64 
 3   Trump             3112 non-null   int64 
 4   2016_total_votes  3112 non-null   int64 
 5   Obama             3112 non-null   int64 
 6   Romney            3112 non-null   int64 
 7   2012_total_votes  3112 non-null   int64 
 8   2010_land_area    3112 non-null   int64 
 9   Density           3112 non-null   int64 
 10  central_outlying  3112 non-null   object
dtypes: int64(8), object(3)
memory usage: 291.8+ KB


Create new column indicating who won the county. To be used as target for modeling.

In [128]:
conditions = [(results['Clinton'] < results['Trump']), (results['Clinton'] > results['Trump'])]
choices = ['Trump', 'Clinton']
results['Target'] = np.select(conditions, choices, default = np.nan)

Drop Clinton and Trump individual columns.

In [129]:
results = results.drop(results.iloc[:, 2:4], axis = 1)

In [130]:
results.head()

,State,County,2016_total_votes,Obama,Romney,2012_total_votes,2010_land_area,Density,central_outlying,Target
0,Alabama,Autauga County,24661,6354,17366,23909,594,93,Central,Trump
1,Alabama,Baldwin County,94090,18329,65772,84988,1590,128,Central,Trump
2,Alabama,Barbour County,10390,5873,5539,11459,885,30,Central,Trump
3,Alabama,Bibb County,8748,2200,6131,8391,623,36,Outlying,Trump
4,Alabama,Blount County,25384,2961,20741,23980,645,89,Outlying,Trump


## ALASKA RESULTS

Data: https://www.thecinyc.com/ 

Open Alaska results file.

In [131]:
alaska_results = pd.read_csv('data/ak_2012_2016.csv')

In [132]:
alaska_results.head()

,Weighted/Muni,Registered Voters,"Clinton, Hillary","Trump, Donald J.",Write-in 60,WtTotal,Unnamed: 6,Weighted/Muni.1,Municode,Registered Voters .1,Total Votes,Johnson/Gray (LIB),Obama/Biden (DEM),Romney/Ryan (REP)
0,Ketchikan Gateway,10512,1966.695802,3451.907138,153.011809,6267.150170,NaN,Ketchikan Gateway,Ketchikan,10257,5905.592707,212.061543,2262.784210,3266.635409
1,Prince of Wales-Hyder,4630,1076.455803,1295.125060,91.039696,2830.673904,NaN,Prince of Wales-Hyder,Prince of Wales-Hyder,4371,2482.062762,76.243288,1298.248629,1045.101630
2,Sitka,7218,2110.994000,1811.544401,115.703622,4427.915868,NaN,Sitka,Sitka,7074,4415.265365,133.089646,2340.002643,1830.694119
3,Petersburg,2741,569.639406,915.365934,50.796562,1706.007455,NaN,Petersburg,Petersburg,2648,1729.880554,59.211545,776.066018,867.459549
4,Wrangell,1731,270.992898,751.941311,19.081913,1124.152497,NaN,Wrangell,Wrangell,1658,1143.064441,26.862207,362.631179,738.686288


Drop unnecessary columns. 

In [133]:
alaska_results = alaska_results.drop(alaska_results.iloc[:, [1, 4, 6, 7, 8, 9, 11]], axis = 1)

Get rid of commas in column names and rename columns.

In [134]:
alaska_results.columns = [col.replace(',', '') for col in alaska_results.columns]

In [135]:
alaska_results = alaska_results.rename(columns = {'Registered Voters': '2016_registered_voters','Weighted/Muni':'County', 'Clinton Hillary':'Clinton', 'Trump Donald J.':'Trump', 'WtTotal':'2016_total_votes', 'Registered Voters .1':'2012_registered_voters', 'Total Votes':'2012_total_votes', 'Obama/Biden (DEM)':'Obama', 'Romney/Ryan (REP)':'Romney'})

Create target column.

In [136]:
conditions = [alaska_results.iloc[:, 2] < alaska_results.iloc[:, 3], alaska_results.iloc[:, 2] > alaska_results.iloc[:, 3]]
choices = ['Trump', 'Clinton']
alaska_results['Target'] = np.select(conditions, choices, default = np.nan)

In [137]:
alaska_results.head()

,County,Clinton Hillary,Trump Donald J.,2016_total_votes,2012_total_votes,Obama,Romney,Target
0,Ketchikan Gateway,1966.695802,3451.907138,6267.150170,5905.592707,2262.784210,3266.635409,Trump
1,Prince of Wales-Hyder,1076.455803,1295.125060,2830.673904,2482.062762,1298.248629,1045.101630,Trump
2,Sitka,2110.994000,1811.544401,4427.915868,4415.265365,2340.002643,1830.694119,Trump
3,Petersburg,569.639406,915.365934,1706.007455,1729.880554,776.066018,867.459549,Trump
4,Wrangell,270.992898,751.941311,1124.152497,1143.064441,362.631179,738.686288,Trump


In [138]:
alaska_results = alaska_results.drop(alaska_results.iloc[:, [1, 2]], axis = 1)

Row 29 is a summed column of all counties. Dropping.

In [139]:
alaska_results.drop(alaska_results.index[29], inplace = True)

Wade Hampton county was renamed Kusilvak.

In [140]:
alaska_results = alaska_results.replace({'Wade Hampton': 'Kusilvak'}, regex = True)

Add state column and sort by county.

In [141]:
alaska_results['State'] = 'Alaska'

In [142]:
alaska_results = alaska_results.sort_values(by = 'County')

In [143]:
alaska_results.head()

,County,2016_total_votes,2012_total_votes,Obama,Romney,Target,State
22,Aleutians East,529.293851,549.375577,234.120530,292.395684,Trump,Alaska
24,Aleutians West,1213.502975,1238.761919,777.428504,426.573343,Trump,Alaska
19,Anchorage,130040.329900,125169.133300,54042.760210,66387.084670,Trump,Alaska
12,Bethel,4892.232820,4810.611592,3425.621480,1151.530057,Trump,Alaska
25,Bristol Bay,453.270615,425.845526,147.147402,251.541638,Trump,Alaska


## Additional Alaska Information

The dataset with Alaska results does not include the additional information that can be found in the results dataset for the rest of the country. Data was pulled from Wikipedia and collected in an excel spreadsheet.  
Data: https://en.wikipedia.org/wiki/List_of_boroughs_and_census_areas_in_Alaska

Open file.

In [144]:
missing_columns = pd.read_csv('data/alaska_missing_columns.csv')

Rename some columns.

In [145]:
missing_columns.rename(columns = {'Land Area': '2010_land_area', 'Metro/Nonmetro status': 'central_outlying'}, inplace = True)

Drop strange extra columns.

In [146]:
missing_columns = missing_columns.drop(missing_columns.iloc[:, 4:6], axis = 1)

Rename Wade Hampton to Kusilvak.

In [147]:
missing_columns.replace('Wade Hampton(kusilvak)', 'Kusilvak', inplace = True)

Change rural to Rural to match other datasets.

In [148]:
missing_columns.replace('rural', 'Rural', inplace = True)

Sort for merge.

In [149]:
missing_columns = missing_columns.sort_values(by = 'Weighted/Muni')

In [150]:
missing_columns.head()

,Weighted/Muni,Density,2010_land_area,central_outlying
0,Aleutians East,0.49,6982,Rural
1,Aleutians West,1.19,4390,Rural
2,Anchorage,170.62,1705,Central
3,Bethel,0.46,40570,Rural
4,Bristol Bay,1.75,504,Rural


## Join Alaska election results with Alaska additional columns.

In [151]:
alaska_total_results = alaska_results.reset_index(drop=True).merge(missing_columns.reset_index(drop=True), left_index=True, right_index=True)

In [152]:
alaska_total_results.head()

,County,2016_total_votes,2012_total_votes,Obama,Romney,Target,State,Weighted/Muni,Density,2010_land_area,central_outlying
0,Aleutians East,529.293851,549.375577,234.120530,292.395684,Trump,Alaska,Aleutians East,0.49,6982,Rural
1,Aleutians West,1213.502975,1238.761919,777.428504,426.573343,Trump,Alaska,Aleutians West,1.19,4390,Rural
2,Anchorage,130040.329900,125169.133300,54042.760210,66387.084670,Trump,Alaska,Anchorage,170.62,1705,Central
3,Bethel,4892.232820,4810.611592,3425.621480,1151.530057,Trump,Alaska,Bethel,0.46,40570,Rural
4,Bristol Bay,453.270615,425.845526,147.147402,251.541638,Trump,Alaska,Bristol Bay,1.75,504,Rural


Look at county columns side by side to double check.

In [153]:
alaska_total_results.iloc[:, [0, 7]]

,County,Weighted/Muni
0,Aleutians East,Aleutians East
1,Aleutians West,Aleutians West
2,Anchorage,Anchorage
3,Bethel,Bethel
4,Bristol Bay,Bristol Bay
5,Denali,Denali
6,Dillingham,Dillingham
7,Fairbanks North Star,Fairbanks North Star
8,Haines,Haines
9,Hoonah-Angoon,Hoonah-Angoon


Drop second county column.

In [154]:
alaska_total_results.drop('Weighted/Muni', axis = 1, inplace = True)

## Join Alaska results with the rest of the country.

In [155]:
total_results = results.append(alaska_total_results)

In [156]:
total_results.head()

,State,County,2016_total_votes,Obama,Romney,2012_total_votes,2010_land_area,Density,central_outlying,Target
0,Alabama,Autauga County,24661.0,6354.0,17366.0,23909.0,594,93.0,Central,Trump
1,Alabama,Baldwin County,94090.0,18329.0,65772.0,84988.0,1590,128.0,Central,Trump
2,Alabama,Barbour County,10390.0,5873.0,5539.0,11459.0,885,30.0,Central,Trump
3,Alabama,Bibb County,8748.0,2200.0,6131.0,8391.0,623,36.0,Outlying,Trump
4,Alabama,Blount County,25384.0,2961.0,20741.0,23980.0,645,89.0,Outlying,Trump


In [157]:
total_results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3141 entries, 0 to 28
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   State             3141 non-null   object 
 1   County            3141 non-null   object 
 2   2016_total_votes  3141 non-null   float64
 3   Obama             3141 non-null   float64
 4   Romney            3141 non-null   float64
 5   2012_total_votes  3141 non-null   float64
 6   2010_land_area    3141 non-null   int64  
 7   Density           3141 non-null   float64
 8   central_outlying  3141 non-null   object 
 9   Target            3141 non-null   object 
dtypes: float64(5), int64(1), object(4)
memory usage: 269.9+ KB


## Race info from 2010 Census

Data: https://data.census.gov/cedsci/table?q=county%20population&tid=DECENNIALPL2020.P1

In [158]:
df_race = pd.read_csv('data/DECENNIALPL2020.P1_data_with_overlays_2021-10-04T202848.csv', error_bad_lines = False, header = None)

Sort out columns and index.

In [159]:
df_race.columns = df_race.iloc[1]

In [160]:
df_race.drop(df_race.index[1], inplace = True)

In [161]:
 df_race.drop(df_race.index[0], inplace = True)

Get rid of columns detailing multi-race breakdown but keep total of multi-race population.

In [162]:
df_race.drop(df_race.iloc[:, 11:73], axis = 1, inplace = True)

Split county and state into two columns and drop old column.

In [163]:
df_race[['County', 'State']] = df_race['Geographic Area Name'].str.split(',', expand=True)

In [164]:
df_race.drop('Geographic Area Name', axis = 1, inplace = True)

Get rid of punctuation.

In [165]:
df_race.columns = df_race.columns.str.strip().str.replace('[^\w\s]', '')

<ipython-input-165-3176a88a0be0>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_race.columns = df_race.columns.str.strip().str.replace('[^\w\s]', '')


In [166]:
def remove_punctuation(x):
    try:
        x = x.str.replace('[^\w\s]','')
    except:
        pass
    return x
df_race = df_race.apply(remove_punctuation)

<ipython-input-166-5daa401cd4a4>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  x = x.str.replace('[^\w\s]','')


Convert race/population info into floats.

In [167]:
df_race[df_race.columns[1:-2]] = df_race[df_race.columns[1:-2]].astype(float)

Chugach and Copper River counties in Alaska were combined between the time of the census and the time of the election. 

In [168]:
chugach = (df_race.loc[df_race['County'] == 'Chugach Census Area']) 

In [169]:
copper_river = (df_race.loc[df_race['County'] == 'Copper River Census Area'])

In [170]:
old_counties = chugach.append(copper_river)
valdez = pd.DataFrame(old_counties.sum(numeric_only = False, axis = 0)).T
df_race.drop(old_counties.index, axis = 0, inplace= True)
df_race = df_race.append(valdez)

In [171]:
df_race = df_race.replace({'0500000US020630500000US02066': 'akcombined', 'Chugach Census AreaCopper River Census Area': 'Valdez Cordova'})
df_race['State'] = df_race['State'].str.replace('Alaska Alaska', 'Alaska')

Drop Puerto Rico because they can not vote in presidential elections.

In [172]:
pr = df_race.loc[df_race['State'].str.contains('Puerto Rico', case=False)]

In [173]:
df_race = df_race.drop(pr.index, axis = 0)

Dropping Kalawoa. It only has 82 residents and is not in other datasets.

In [174]:
Kalawao = df_race.loc[df_race['County'].str.contains('Kalawao', case=False)]

In [175]:
df_race = df_race.drop(Kalawao.index, axis = 0)

Rename annoying columns.

In [176]:
df_race.rename(columns = {'Total': 'total_pop', 'TotalPopulation of one race': 'total_pop_one_race', 'TotalPopulation of one raceWhite alone':'pop_white', 'TotalPopulation of one raceBlack or African American alone': 'pop_african_american', 'TotalPopulation of one raceAmerican Indian and Alaska Native alone': 'pop_native', 'TotalPopulation of one raceAsian alone': 'pop_asian', 'TotalPopulation of one raceNative Hawaiian and Other Pacific Islander alone': 'pop_islander', 'TotalPopulation of one raceSome Other Race alone': 'pop_other', 'TotalPopulation of two or more races': 'total_pop_two_races'}, inplace = True)

Sort and reset index for joining.

In [177]:
df_race = df_race.sort_values(by = ['State', 'County']).reset_index(drop = True)

## Poverty census info

Data: https://data.census.gov/cedsci/all?q=county%20population

In [178]:
df = pd.read_csv('data/poverty.csv')

Sort out columns and index.

In [179]:
df.columns = df.iloc[0]

In [180]:
df.drop(df.index[0], inplace = True)

In [181]:
df = df.reset_index(drop = True)

Remove columns that contain the word 'Bound'. These are not needed.

In [182]:
df = df.loc[:, ~df.columns.str.contains('Bound')]

Remove columns that are percentages.

In [183]:
df = df.loc[:, ~df.columns.str.contains('Percent')]

Locate DC and change County FIPS so it isn't dropped during the next step.

In [184]:
df.loc[(df['County FIPS'] == '000') & (df['Name'] == 'District of Columbia')]

,State FIPS,County FIPS,Postal,Name,Poverty Estimate All Ages,Poverty Estimate Under Age 18,Poverty Estimate Ages 5-17,Median Household Income,Poverty Estimate Ages 0-4
328,11,000,DC,District of Columbia,"107,279","31,147","20,872","60,729","9,786"


In [185]:
df.iloc[327:330, :]

,State FIPS,County FIPS,Postal,Name,Poverty Estimate All Ages,Poverty Estimate Under Age 18,Poverty Estimate Ages 5-17,Median Household Income,Poverty Estimate Ages 0-4
327,10,005,DE,Sussex County,"26,924","9,501","6,123","48,582",NaN
328,11,000,DC,District of Columbia,"107,279","31,147","20,872","60,729","9,786"
329,11,001,DC,District of Columbia,"107,279","31,147","20,872","60,729",NaN


Nevermind. DC is in there alone and summed (as if it was a state). Just drop all County FIPS 000.

Drop all rows with county FIPS 000. These are just states summed. We don't need them. Might be useful to look at for any missing information later though.

In [186]:
drop = df.loc[(df['County FIPS'] == '000')]

In [187]:
df = df.drop(drop.index, axis = 0)

Drop Kalawao. No election results.

In [188]:
Kalawao = (pd.DataFrame(df.loc[561]))
df = df.drop(Kalawao)

Change state abbreviations to full names.

In [189]:
us_state_abbrev = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming',
}

In [190]:
df.Postal = df.Postal.map(us_state_abbrev)

Drop unwanted columns.

In [191]:
df = df.drop(df.iloc[:, [0, 1, 6, 8]], axis = 1)

Chugach and Copper River are already combined into Valdez-Cordova so no need to change that. Renaming Wade Hampton to Kusilvak to match results.

In [192]:
df.loc[95]

0
Postal                                             Alaska
Name                             Wade Hampton Census Area
Poverty Estimate All Ages                           2,530
Poverty Estimate Under Age 18                       1,403
Median Household Income                            30,883
Name: 95, dtype: object

In [193]:
df.loc[95]['Name'] = 'Kusilvak'

Remove punctuation.

In [194]:
def remove_punctuation(x):
    try:
        x = x.str.replace('[^\w\s]','')
    except:
        pass
    return x
df = df.apply(remove_punctuation)

<ipython-input-194-8e12971db89d>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  x = x.str.replace('[^\w\s]','')


Rename columns.

In [195]:
df.columns = ['state', 'county', 'poverty_total', 'poverty_under_18', 'median_household_income']

Sort and reset index.

In [196]:
df = df.sort_values(by = ['state', 'county']).reset_index(drop = True)

Drop empty rows at the bottom.

In [197]:
drop = df.loc[3142:]

In [198]:
df = df.drop(drop.index, axis = 0)

Change column types.

In [199]:
df[df.columns[2:]] = df[df.columns[2:]].astype(float)

Bedford, Virginia was a city and a county during this census. They were not separated for the elections. 

In [200]:
bedfords = df.loc[2828:2829]

In [201]:
new_bedford = pd.DataFrame(bedfords.sum(numeric_only = False, axis = 0)).T

In [202]:
new_bedford['state'] = 'Virginia'
new_bedford['county'] = 'Bedford'
new_bedford['median_household_income'] = 56043 #pulled this from usa.com

In [203]:
df.drop(bedfords.index, axis = 0, inplace = True)

In [204]:
df = df.append(new_bedford)

Fill state name for DC.

In [205]:
df.loc[3141, 'state'] = 'District of Columbia'

In [206]:
df.loc[3141]

state                      District of Columbia
county                     District of Columbia
poverty_total                          107279.0
poverty_under_18                        31147.0
median_household_income                 60729.0
Name: 3141, dtype: object

In [207]:
df.loc[df.county == 'District of Columbia']

,state,county,poverty_total,poverty_under_18,median_household_income
3141,District of Columbia,District of Columbia,107279.0,31147.0,60729.0


Rename Shannon County, SD to Oglala Lakota County to match other datasets.

In [208]:
df.loc[2415]

state                        South Dakota
county                     Shannon County
poverty_total                      6428.0
poverty_under_18                   2631.0
median_household_income           27307.0
Name: 2415, dtype: object

Change datatype of some columns.

In [209]:
df[df.columns[2:]] = df[df.columns[2:]].astype(float)

In [210]:
poverty_df = df

In [211]:
poverty_df.head()

,state,county,poverty_total,poverty_under_18,median_household_income
0,Alabama,Autauga County,6459.0,2530.0,53049.0
1,Alabama,Baldwin County,24056.0,8357.0,47618.0
2,Alabama,Barbour County,6098.0,2145.0,33074.0
3,Alabama,Bibb County,4316.0,1448.0,35472.0
4,Alabama,Blount County,9358.0,3356.0,42906.0


## Merge race and poverty census data with results

Sort for merge.

In [212]:
poverty_df = poverty_df.sort_values(by = ['state', 'county']).reset_index(drop = True)

Sort for merge.

In [213]:
df_race = df_race.sort_values(by = ['State', 'County']).reset_index(drop = True)

Sort for merge.

In [214]:
total_results = total_results.sort_values(by = ['State', 'County']).reset_index(drop = True)

Merge race dataset and total results dataset.

In [215]:
df_race_results = df_race.reset_index(drop=True).merge(total_results.reset_index(drop=True), left_index=True, right_index=True)

In [216]:
df_race_results.loc[:, ['County_x', 'County_y', ]].tail()

,County_x,County_y
3136,Sweetwater County,Sweetwater County
3137,Teton County,Teton County
3138,Uinta County,Uinta County
3139,Washakie County,Washakie County
3140,Weston County,Weston County


Merge poverty with results and race.

In [217]:
df_poverty_race_results = df_race_results.reset_index(drop=True).merge(poverty_df.reset_index(drop=True), left_index=True, right_index=True)

In [218]:
df_poverty_race_results.loc[:, ['County_x', 'County_y', 'county']]

,County_x,County_y,county
0,Autauga County,Autauga County,Autauga County
1,Baldwin County,Baldwin County,Baldwin County
2,Barbour County,Barbour County,Barbour County
3,Bibb County,Bibb County,Bibb County
4,Blount County,Blount County,Blount County
5,Bullock County,Bullock County,Bullock County
6,Butler County,Butler County,Butler County
7,Calhoun County,Calhoun County,Calhoun County
8,Chambers County,Chambers County,Chambers County
9,Cherokee County,Cherokee County,Cherokee County


In [2]:
df_all.head()

NameError: name 'df_all' is not defined

In [219]:
df_all = df_poverty_race_results

In [220]:
dump(df_all, open('df_all.pkl', 'wb'))